# Your First RAG Engine Deployment: A Complete Walkthrough
## Learn by Doing: From Zero to Production

**Welcome!** This notebook will guide you through your very first deployment of RAG Engine, explaining every step in detail. By the end, you'll understand not just *what* to do, but *why* you're doing it.

**What you'll learn:**
1. How containers work (with hands-on examples)
2. How to build your first Docker image
3. How to deploy with Docker Compose
4. How to verify everything is working
5. How to troubleshoot common issues

**Prerequisites:**
- Basic command line knowledge
- Docker and Docker Compose installed
- Curiosity and patience!

**Estimated time:** 2-3 hours

Let's begin! 🚀

## Part 1: Understanding What We're Building

Before we deploy anything, let's understand what RAG Engine is and why we need multiple services.

### What is RAG?

**RAG = Retrieval-Augmented Generation**

Imagine you're writing an essay and you can:
1. Search your library of books for relevant information
2. Read the relevant sections
3. Write your essay using that information

That's what RAG does with AI:
- **Retrieval**: Find relevant documents from your knowledge base
- **Augmented**: Use those documents to provide context
- **Generation**: Generate AI responses based on the retrieved context

**Why is this powerful?**
- AI can answer questions about YOUR specific documents
- Responses are grounded in facts (not hallucinations)
- You can update the knowledge base anytime

**Real-world example:**
Upload your company's policy documents → Ask "What's our remote work policy?" → Get accurate answer from your documents

### RAG Engine Architecture

Our deployment consists of 4 main services:

```
┌─────────────────────────────────────────────────────────────┐
│                    RAG Engine Stack                         │
│                                                             │
│  ┌──────────────┐                                          │
│  │     API      │  ← FastAPI web application               │
│  │   (Python)   │     Handles HTTP requests                │
│  └──────┬───────┘                                          │
│         │                                                   │
│  ┌──────┴──────┬──────────┬──────────────┐                │
│  ▼            ▼          ▼              ▼                │
│ ┌─────┐  ┌────────┐ ┌──────────┐  ┌──────────┐          │
│ │PostgreSQL│ │  Redis  │ │  Qdrant  │  │  Nginx   │          │
│ │(Database)│ │ (Cache) │ │ (Vectors)│  │ (Proxy)  │          │
│ └─────┘  └────────┘ └──────────┘  └──────────┘          │
│    ↓          ↓          ↓                                 │
│  User       Session   Document                           │
│  Data        Cache    Embeddings                         │
└─────────────────────────────────────────────────────────────┘
```

**Why separate services?**

1. **API Service (FastAPI)**
   - Handles all HTTP requests
   - Processes uploaded documents
   - Manages search queries
   - Communicates with other services

2. **PostgreSQL (Database)**
   - Stores user information
   - Stores document metadata
   - Stores query history
   - ACID compliant (reliable)

3. **Redis (Cache)**
   - Caches frequently accessed data
   - Stores session information
   - Very fast (in-memory)
   - Reduces database load

4. **Qdrant (Vector Database)**
   - Stores document embeddings (vectors)
   - Performs similarity search
   - Finds relevant documents quickly
   - Specialized for AI/ML workloads

5. **Nginx (Reverse Proxy)**
   - SSL termination (HTTPS)
   - Load balancing
   - Static file serving
   - Security layer

## Part 2: Checking Your Environment

Before we start, let's verify that your environment is ready.

In [ ]:
import subprocess
import sys

def check_prerequisites():
    """Check if required tools are installed"""
    
    print("=== Checking Prerequisites ===\n")
    
    checks = {
        'docker': 'Docker',
        'docker-compose': 'Docker Compose',
        'curl': 'cURL',
        'git': 'Git'
    }
    
    all_good = True
    
    for cmd, name in checks.items():
        try:
            result = subprocess.run([cmd, '--version'], 
                                  capture_output=True, 
                                  text=True, 
                                  timeout=5)
            if result.returncode == 0:
                version = result.stdout.strip().split('\n')[0][:50]
                print(f"✅ {name}: {version}")
            else:
                print(f"❌ {name}: Command failed")
                all_good = False
        except FileNotFoundError:
            print(f"❌ {name}: Not installed")
            all_good = False
        except Exception as e:
            print(f"⚠️  {name}: Error checking - {e}")
    
    print()
    if all_good:
        print("🎉 All prerequisites are satisfied!")
        print("\nYou can proceed with the deployment.")
        return True
    else:
        print("⚠️  Some prerequisites are missing.")
        print("\nPlease install the missing tools:")
        print("  • Docker: https://docs.docker.com/get-docker/")
        print("  • Docker Compose: Usually included with Docker Desktop")
        return False

# Run the check
ready = check_prerequisites()
if not ready:
    print("\n⚠️  Please install the missing prerequisites before continuing.")
    print("Once installed, run this cell again.")

## Part 3: Creating Our Project Structure

Let's organize our files properly. Good organization makes everything easier.

In [ ]:
import os
from pathlib import Path

def create_project_structure():
    """Create the directory structure for our project"""
    
    # Define our project root
    project_root = Path('./rag-engine-deployment')
    
    # Create directories
    directories = [
        project_root,
        project_root / 'data' / 'postgres',
        project_root / 'data' / 'redis',
        project_root / 'data' / 'qdrant',
        project_root / 'data' / 'documents',
        project_root / 'logs',
        project_root / 'ssl',
        project_root / 'app'
    ]
    
    print("=== Creating Project Structure ===\n")
    
    for directory in directories:
        directory.mkdir(parents=True, exist_ok=True)
        print(f"📁 {directory}/")
    
    print()
    print("✅ Project structure created!")
    print(f"\n📂 Project location: {project_root.absolute()}")
    
    return project_root

# Create the structure
project_dir = create_project_structure()
print(f"\nWe'll be working in: {project_dir.absolute()}")

### Understanding the Directory Structure

```
rag-engine-deployment/
├── app/                   ← Application code (we'll create this)
├── data/
│   ├── postgres/         ← Database files (persisted)
│   ├── redis/            ← Cache data (persisted)
│   ├── qdrant/           ← Vector database (persisted)
│   └── documents/        ← Uploaded documents
├── logs/                 ← Application logs
├── ssl/                  ← SSL certificates (for HTTPS)
├── docker-compose.yml    ← Docker Compose configuration
├── Dockerfile            ← API container definition
├── .env                  ← Environment variables (secrets)
└── nginx.conf            ← Nginx configuration
```

**Why this structure?**

1. **Separation of concerns**: Code, data, and config are separate
2. **Persistence**: `data/` directory survives container restarts
3. **Version control**: Only code and config are committed
4. **Security**: Secrets in `.env` (never committed)
5. **Logs**: Centralized in `logs/` for easy debugging

## Part 4: Creating Our Application

Let's create a simple RAG Engine application. We'll build it step by step.

### Step 1: The Application Code

We'll create a minimal but functional FastAPI application.

In [ ]:
# Create the main application file
app_code = '''
"""
RAG Engine - A simple Retrieval-Augmented Generation API

This is a minimal but complete implementation showing:
- FastAPI web framework
- Health check endpoints
- Document upload and search
- Integration with PostgreSQL, Redis, and Qdrant
"""

from fastapi import FastAPI, HTTPException, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional
import os
import hashlib
from datetime import datetime

# Initialize FastAPI application
app = FastAPI(
    title="RAG Engine",
    description="A simple RAG implementation for learning Docker deployment",
    version="1.0.0"
)

# Enable CORS (Cross-Origin Resource Sharing)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, specify exact origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configuration from environment variables
ENVIRONMENT = os.getenv("ENVIRONMENT", "development")
DEBUG = os.getenv("DEBUG", "true").lower() == "true"


# ============================================================================
# Data Models
# ============================================================================

class SearchQuery(BaseModel):
    """Model for search queries"""
    query: str
    top_k: int = 5
    
    class Config:
        json_schema_extra = {
            "example": {
                "query": "What is machine learning?",
                "top_k": 5
            }
        }

class SearchResult(BaseModel):
    """Model for search results"""
    id: str
    content: str
    score: float
    metadata: dict

class DocumentMetadata(BaseModel):
    """Model for document metadata"""
    filename: str
    size: int
    uploaded_at: str
    status: str


# ============================================================================
# Health Check Endpoints
# ============================================================================

@app.get("/")
async def root():
    """Root endpoint - API information"""
    return {
        "name": "RAG Engine",
        "version": "1.0.0",
        "environment": ENVIRONMENT,
        "docs": "/docs",
        "health": "/health"
    }


@app.get("/health")
async def health_check():
    """
    Health check endpoint for monitoring and load balancers.
    
    Returns 200 if the API is healthy.
    """
    try:
        # In a real app, check database connections here
        # For now, just return success
        return {
            "status": "healthy",
            "timestamp": datetime.utcnow().isoformat(),
            "environment": ENVIRONMENT,
            "version": "1.0.0"
        }
    except Exception as e:
        raise HTTPException(status_code=503, detail=str(e))


@app.get("/ready")
async def readiness_check():
    """
    Readiness check - returns 200 when ready to accept traffic.
    
    Kubernetes uses this to know when to send requests.
    """
    return {
        "status": "ready",
        "timestamp": datetime.utcnow().isoformat()
    }


# ============================================================================
# Document Management Endpoints
# ============================================================================

# In-memory storage for demo (in production, use database)
documents = {}


@app.post("/api/v1/documents/upload")
async def upload_document(file: UploadFile = File(...)):
    """
    Upload a document for indexing.
    
    In a real implementation, this would:
    1. Save the file to storage
    2. Extract text from the document
    3. Generate embeddings (vectors)
    4. Store in vector database (Qdrant)
    """
    try:
        # Read file content
        content = await file.read()
        
        # Generate document ID
        doc_id = hashlib.md5(content).hexdigest()[:12]
        
        # Store document metadata
        documents[doc_id] = {
            "id": doc_id,
            "filename": file.filename,
            "size": len(content),
            "content": content.decode('utf-8', errors='ignore')[:1000],  # Truncated for demo
            "uploaded_at": datetime.utcnow().isoformat(),
            "status": "indexed"
        }
        
        return {
            "id": doc_id,
            "filename": file.filename,
            "size": len(content),
            "status": "uploaded",
            "message": "Document uploaded successfully (demo mode)"
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Upload failed: {str(e)}")


@app.get("/api/v1/documents/{doc_id}")
async def get_document(doc_id: str):
    """Get document metadata by ID"""
    if doc_id not in documents:
        raise HTTPException(status_code=404, detail="Document not found")
    
    doc = documents[doc_id]
    return {
        "id": doc["id"],
        "filename": doc["filename"],
        "size": doc["size"],
        "uploaded_at": doc["uploaded_at"],
        "status": doc["status"]
    }


# ============================================================================
# Search Endpoints
# ============================================================================


@app.post("/api/v1/search", response_model=List[SearchResult])
async def search_documents(query: SearchQuery):
    """
    Search for documents using a text query.
    
    In a real implementation, this would:
    1. Convert query to embedding vector
    2. Search Qdrant for similar vectors
    3. Return most relevant documents
    
    For this demo, we do simple text matching.
    """
    try:
        results = []
        query_lower = query.query.lower()
        
        # Simple keyword matching (demo only)
        for doc_id, doc in documents.items():
            content = doc.get("content", "").lower()
            
            # Calculate simple relevance score
            score = 0
            for word in query_lower.split():
                if word in content:
                    score += content.count(word)
            
            if score > 0:
                results.append(SearchResult(
                    id=doc_id,
                    content=doc["content"][:200] + "..." if len(doc["content"]) > 200 else doc["content"],
                    score=min(score / 10, 1.0),  # Normalize to 0-1
                    metadata={
                        "filename": doc["filename"],
                        "uploaded_at": doc["uploaded_at"]
                    }
                ))
        
        # Sort by score (highest first)
        results.sort(key=lambda x: x.score, reverse=True)
        
        # Return top_k results
        return results[:query.top_k]
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Search failed: {str(e)}")


# ============================================================================
# Application Startup
# ============================================================================
